In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


sync_confirmed_list_train_all = [[23,27,28,29,30,32,33,37,38,39,40,41,42,45,46,50,53,56,57,59,62,63,66,67,68,69,71,72,73],
                                [9,10,11,14,15,17,18,24,25,29,31,39,40,41,42,50,53,56,57,59,62,63,66,67,68,69,71,72,73],
                                [9,10,11,14,15,17,18,23,24,25,27,28,30,31,32,33,37,38,45,46,56,57,62,67,68,69,71,72,73],
                                [9,10,11,14,15,17,18,23,24,25,27,28,29,30,31,32,33,37,38,39,40,41,42,45,46,50,53,59,63,66]]

sync_confirmed_list_test_all = [[9,10,11,14,15,17,18,24,25,31],
                               [23,27,28,30,32,33,37,38,45,46],
                               [29,39,40,41,42,50,53,59,63,66],
                               [56,57,62,67,68,69,71,72,73]]



# sync_confirmed_list_train_all = [[23,27,28,29,30,32,33,37,38,39,40,41,42,45,46,50,53,56,57,59,62,63,66,67],[50,53,56,57,59,62,63,66,67,68,69,71,72,73,9,10,11,14,15,17,18,24,25,31],[23,27,28,29,30,32,33,37,38,39,40,41,72,73,9,10,11,14,15,17,18,24,25,31],[23,27,29,30,33,37,39,40,42,45,50,53,57,59,63,66,68,69,72,73,10,11,15,17]]

# sync_confirmed_list_test_all = [[68,69,71,72,73,9,10,11,14,15,17,18,24,25,31],[23,27,28,29,30,32,33,37,38,39,40,41,42,45,46],[42,45,46,50,53,56,57,59,62,63,66,67,68,69,71],[28,32,38,41,46,56,62,67,71,9,14,24,25,31,18]]

sync_confirmed_list_train = sync_confirmed_list_train_all[3]
sync_confirmed_list_test = sync_confirmed_list_test_all[3]

tolerance_degree = 0

# Participant_file_name = 'ML-PersRef_ML_Final_dataset/datapoints_rawdata_first_middle_points_with_building_width_outlier_removed_'
#Participant_file_name = 'ML-PersRef_ML_Final_dataset/datapoints_rawdata_first_middle_points_with_building_width_splitted_near_'
#Participant_file_name = 'ML-PersRef_ML_Final_dataset/datapoints_rawdata_first_middle_points_with_building_width_splitted_far_'

Participant_file_name = 'ML-PersRef_ML_Final_dataset/datapoints_rawdata_first_middle_points_with_building_width_'


# sync_confirmed_list_train, sync_confirmed_list_test = train_test_split(sync_confirmed_list, test_size=0.38, random_state=42)
print(sync_confirmed_list_train)
print(sync_confirmed_list_test)

dataset = []
dataset_test = []

for z in sync_confirmed_list_train:
    dataset.append(pd.read_csv(Participant_file_name + str(z)+'.csv'))
    
for j in sync_confirmed_list_test:
    dataset_test.append(pd.read_csv(Participant_file_name + str(j)+'.csv'))
    
dataset = pd.concat(dataset).reset_index(drop=True)
dataset_test = pd.concat(dataset_test).reset_index(drop=True)
dataset

In [ ]:
dataset_test

In [ ]:
regressor = SVR(kernel='rbf', epsilon=2)
#regressor = SVR(kernel='rbf', epsilon=5)


xtrain = dataset.iloc[:,1:5].values.astype(float)
ytrain = dataset.iloc[:,6].values.astype(float)
ytrain_width = dataset.iloc[:,7].values.astype(float)

sc_xtrain = StandardScaler()
#sc_xtrain = MinMaxScaler()

xtrain = sc_xtrain.fit_transform(xtrain)

xtest = dataset_test.iloc[:,1:5].values.astype(float)
ytest = dataset_test.iloc[:,6].values.astype(float)
ytest_width = dataset_test.iloc[:,7].values.astype(float)

sc_xtest = StandardScaler()
#sc_xtest = MinMaxScaler()
xtest = sc_xtest.fit_transform(xtest)

print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

xtrain_eye_gaze = xtrain[:,-1].reshape(-1, 1)
xtest_eye_gaze = xtest[:,-1].reshape(-1, 1)
xtrain_pointing = xtrain[:,0:3]
xtest_pointing = xtest[:,0:3]
ytrain = ytrain
ytest = ytest



#print(xtrain, xtest, xtrain_eye_gaze, xtest_eye_gaze, xtrain_pointing, xtest_pointing, ytrain, ytest)
print(xtrain.shape, xtest.shape, xtrain_eye_gaze.shape, xtest_eye_gaze.shape, xtrain_pointing.shape, xtest_pointing.shape, ytrain.shape, ytest.shape)

print(np.percentile(ytest_width,50))
print(ytest_width.mean())

print("========================")
print("---------train----------")

regressor.fit(xtrain,ytrain)

print('-----------------')
print(regressor.support_vectors_.shape)
pred=regressor.predict(xtrain)
print(r2_score(ytrain,pred))
print(mean_squared_error(ytrain, pred))
print('-----------------')

print('================================================')
#print(ytrain,pred)
#print(abs(ytrain-pred))
#print(ytrain_width)
relative_width = abs(ytrain-pred)/(ytrain_width + tolerance_degree)
#print(relative_width)
print(relative_width.mean())
print(relative_width.std())
print('================================================')

xtrain_fuse = regressor.support_vectors_
ytrain_fuse = ytrain[regressor.support_]

regressor.fit(xtrain_pointing,ytrain)
print(regressor.support_vectors_.shape)
pred=regressor.predict(xtrain_pointing)
print(r2_score(ytrain,pred))
print(mean_squared_error(ytrain, pred))
print('-----------------')

print('================================================')
#print(ytrain,pred)
#print(abs(ytrain-pred))
#print(ytrain_width)
relative_width = abs(ytrain-pred)/(ytrain_width + tolerance_degree)
#print(relative_width)
print(relative_width.mean())
print(relative_width.std())
print('================================================')

xtrain_pointing = regressor.support_vectors_
ytrain_pointing = ytrain[regressor.support_]

regressor.fit(xtrain_eye_gaze,ytrain)
print(regressor.support_vectors_.shape)
pred=regressor.predict(xtrain_eye_gaze)
print(r2_score(ytrain,pred))
print(mean_squared_error(ytrain, pred))
print('-----------------')

print('================================================')
#print(ytrain,pred)
#print(abs(ytrain-pred))
#print(ytrain_width)
relative_width = abs(ytrain-pred)/(ytrain_width + tolerance_degree)
#print(relative_width)
print(relative_width.mean())
print(relative_width.std())
print('================================================')

xtrain_eye_gaze = regressor.support_vectors_
ytrain_eye_gaze = ytrain[regressor.support_]

print("========================")
print("----------test----------")

regressor.fit(xtrain_fuse,ytrain_fuse)
print('-----------------')
print(regressor.support_vectors_.shape)
pred=regressor.predict(xtest)
print(r2_score(ytest,pred))
print(mean_squared_error(ytest, pred))
print('-----------------')

print('================================================')
#print(ytest,pred)
#print(abs(ytest-pred))
#print(ytest_width)
relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
#print(relative_width)
print(relative_width.mean())
print(relative_width.std())
print('================================================')

regressor.fit(xtrain_pointing,ytrain_pointing)
print(regressor.support_vectors_.shape)
pred=regressor.predict(xtest_pointing)
print(r2_score(ytest,pred))
print(mean_squared_error(ytest, pred))
print('-----------------')

print('================================================')
#print(ytest,pred)
#print(abs(ytest-pred))
#print(ytest_width)
relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
#print(relative_width)
print(relative_width.mean())
print(relative_width.std())
print('================================================')

regressor.fit(xtrain_eye_gaze,ytrain_eye_gaze)
print(regressor.support_vectors_.shape)
pred=regressor.predict(xtest_eye_gaze)
print(r2_score(ytest,pred))
print(mean_squared_error(ytest, pred))
print('-----------------')

print('================================================')
#print(ytest,pred)
#print(abs(ytest-pred))
#print(ytest_width)
relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
#print(relative_width)
print(relative_width.mean())
print(relative_width.std())
print('================================================')

print(xtrain.shape, xtrain_fuse.shape, xtrain_pointing.shape, xtrain_eye_gaze.shape,  ytrain.shape, ytrain_fuse.shape, ytrain_pointing.shape, ytrain_eye_gaze.shape)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(15,10))
#ax1.set_title('Building Half-width Box Plot', fontsize = 20.0)
ax1.boxplot(ytest_width, showfliers=False,showmeans=True, meanline=True)
ax1.get_xaxis().set_visible(False)
ax1.set_ylabel('Building Half-width in degrees', fontsize = 20.0) # Y label
ax1.tick_params(axis='both', which='major', labelsize=20)

In [ ]:
def personalisation_svr_test_train(participant_no,sc_xtrain,sc_xtest,xtrain_fuse,ytrain_fuse,xtrain_pointing,ytrain_pointing,xtrain_eye_gaze,ytrain_eye_gaze,sample_weight_value):

    
#     xtrain = (dataset.iloc[:,1:5].values.astype(float))
#     ytrain = (dataset.iloc[:,-1].values.astype(float))
    
    print('P',participant_no)
    dataset_test_participant = pd.read_csv(Participant_file_name + str(participant_no)+'.csv')


    xtest = dataset_test_participant.iloc[:,1:5].values.astype(float)
    ytest = dataset_test_participant.iloc[:,6:8].values.astype(float)

    #print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)
    #print('------------------------------')

    
    from sklearn.model_selection import train_test_split

    xtrain_participant,xtest, ytrain_participant_both , ytest_both = train_test_split(xtest, ytest, test_size=0.5, random_state=42)
    
    
    
    xtrain_participant = sc_xtrain.fit_transform(xtrain_participant)
    xtrain_eye_gaze_participant = xtrain_participant[:,-1].reshape(-1, 1)
    xtrain_pointing_participant = xtrain_participant[:,0:3]
    ytrain_participant = ytrain_participant_both[:,0]


    xtest = sc_xtest.fit_transform(xtest)
    xtest_eye_gaze = xtest[:,-1].reshape(-1, 1)
    xtest_pointing = xtest[:,0:3]
    ytest = ytest_both[:,0]
    ytest_width = ytest_both[:,1]


#     #############################################################################
#     #get before personalization
    
#     regressor.fit(xtrain_fuse,ytrain_fuse)
#     #print('-----------------')
#     pred=regressor.predict(xtest)
#     print(mean_squared_error(ytest, pred))
#     #print('-----------------')

#     regressor.fit(xtrain_pointing,ytrain_pointing)
#     pred=regressor.predict(xtest_pointing)
#     print(mean_squared_error(ytest, pred))
#     #print('-----------------')


#     regressor.fit(xtrain_eye_gaze,ytrain_eye_gaze)
#     pred=regressor.predict(xtest_eye_gaze)
#     print(mean_squared_error(ytest, pred))
#     #print('-----------------')    
#     #############################################################################  

    
    

    #add participants to training
    
    
    xtrain_fuse = np.concatenate((xtrain_fuse, xtrain_participant),axis=0)
    ytrain_fuse = np.concatenate((ytrain_fuse, ytrain_participant),axis=0)
    
    xtrain_pointing = np.concatenate((xtrain_pointing, xtrain_pointing_participant),axis=0)
    ytrain_pointing = np.concatenate((ytrain_pointing, ytrain_participant),axis=0)
    
    xtrain_eye_gaze = np.concatenate((xtrain_eye_gaze, xtrain_eye_gaze_participant),axis=0)
    ytrain_eye_gaze = np.concatenate((ytrain_eye_gaze, ytrain_participant),axis=0)

    
    
    #############################################################################
    #get after personalization
    
    

    sample_weight_fuse = 1 * np.ones((len(ytrain_fuse)))
    sample_weight_fuse[-len(ytrain_participant):] *= sample_weight_value
    
    sample_weight_point = 1 * np.ones((len(ytrain_pointing)))
    sample_weight_point[-len(ytrain_participant):] *= sample_weight_value

    sample_weight_gaze = 1 * np.ones((len(ytrain_eye_gaze)))
    sample_weight_gaze[-len(ytrain_participant):] *= sample_weight_value
    
#     print('--------=======---------')
#     print(sample_weight_fuse.shape,sample_weight_point.shape,sample_weight_gaze.shape)
#     #print(sample_weight_fuse,sample_weight_point,sample_weight_gaze)
#     print('--------=======---------')
    
    
    
    regressor.fit(xtrain_fuse,ytrain_fuse,sample_weight=sample_weight_fuse)
    #print('-----------------')
    pred=regressor.predict(xtest)
    print(mean_squared_error(ytest, pred))
    #print('-----------------')
    
    print('================================================')
    #print(ytest,pred)
    #print(abs(ytest-pred))
    #print(ytest_width)
    relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
    #print(relative_width)
    print(relative_width.mean())
    print(relative_width.std())
    print('================================================')

    regressor.fit(xtrain_pointing,ytrain_pointing,sample_weight=sample_weight_point)
    pred=regressor.predict(xtest_pointing)
    print(mean_squared_error(ytest, pred))
    #print('-----------------')

    print('================================================')
    #print(ytest,pred)
    #print(abs(ytest-pred))
    #print(ytest_width)
    relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
    #print(relative_width)
    print(relative_width.mean())
    print(relative_width.std())
    print('================================================')

    regressor.fit(xtrain_eye_gaze,ytrain_eye_gaze,sample_weight=sample_weight_gaze)
    pred=regressor.predict(xtest_eye_gaze)
    print(mean_squared_error(ytest, pred))
    
    print('================================================')
    #print(ytest,pred)
    #print(abs(ytest-pred))
    #print(ytest_width)
    relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
    #print(relative_width)
    print(relative_width.mean())
    print(relative_width.std())
    print('================================================')
    
    #print('-----------------')     
    #############################################################################
    
    
    #print(xtrain.shape, xtest.shape, xtrain_eye_gaze.shape, xtest_eye_gaze.shape, xtrain_pointing.shape, xtest_pointing.shape, ytrain.shape, ytest.shape)
    #print('------------------------------')
    
    #print('------=====begin inside loop=====------')
    
    
    MSE_fusion = []
    MSE_pointing = []
    MSE_Gaze = []
    
    RMDE_fusion = []
    RMDE_pointing = []
    RMDE_Gaze = []
    
    for participant_no_inside in sync_confirmed_list_test:
    
        #print('P',participant_no_inside)
        dataset_test_participant_2 = pd.read_csv(Participant_file_name+ str(participant_no_inside)+'.csv')


        xtest = dataset_test_participant_2.iloc[:,1:5].values.astype(float)
        ytest = dataset_test_participant_2.iloc[:,6:8].values.astype(float)
        ssss, xtest, ssss , ytest = train_test_split(xtest, ytest, test_size=0.5, random_state=42)
        sc_xtest = StandardScaler()
        #sc_xtest = MinMaxScaler()
        xtest = sc_xtest.fit_transform(xtest)


        xtest_eye_gaze = xtest[:,-1].reshape(-1, 1)
        xtest_pointing = xtest[:,0:3]
        ytest_width = ytest[:,1]
        ytest = ytest[:,0]
        


        regressor.fit(xtrain_fuse,ytrain_fuse,sample_weight=sample_weight_fuse)
        pred=regressor.predict(xtest)
        MSE_fusion.append(mean_squared_error(ytest, pred))
        relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
        RMDE_fusion.append(relative_width.mean())
        
        regressor.fit(xtrain_pointing,ytrain_pointing,sample_weight=sample_weight_point)
        pred=regressor.predict(xtest_pointing)
        MSE_pointing.append(mean_squared_error(ytest, pred))
        relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
        RMDE_pointing.append(relative_width.mean())

        regressor.fit(xtrain_eye_gaze,ytrain_eye_gaze,sample_weight=sample_weight_gaze)
        pred=regressor.predict(xtest_eye_gaze)
        MSE_Gaze.append(mean_squared_error(ytest, pred))
        relative_width = abs(ytest-pred)/(ytest_width + tolerance_degree)
        RMDE_Gaze.append(relative_width.mean())        

    appended_MSE_fusion.append(MSE_fusion)
    appended_MSE_pointing.append(MSE_pointing)
    appended_MSE_Gaze.append(MSE_Gaze)
    
    appended_RMDE_fusion.append(RMDE_fusion)
    appended_RMDE_pointing.append(RMDE_pointing)
    appended_RMDE_Gaze.append(RMDE_Gaze)
    
    #print('------=====end inside loop=====------')
    #print('------=====end Participant=====------')
    print('------=========================------')

In [ ]:
sample_weight_values = [15]

average_fusion_MSE_weights = []
average_fusion_MSE_non_weights_total = []

average_fusion_RMDE_weights = []
average_fusion_RMDE_non_weights_total = []

for sample_weight_value in sample_weight_values:

    average_fusion_MSE_per_weight = []
    average_fusion_RMDE_per_weight = []

    average_fusion_MSE_others_total = []
    average_fusion_RMDE_others_total = []
    
    
    appended_MSE_fusion = []
    appended_MSE_pointing = []
    appended_MSE_Gaze = []

    appended_RMDE_fusion = []
    appended_RMDE_pointing = []
    appended_RMDE_Gaze = []

    for i in sync_confirmed_list_test:
        personalisation_svr_test_train(i,sc_xtrain,sc_xtest,xtrain_fuse,ytrain_fuse,xtrain_pointing,ytrain_pointing,xtrain_eye_gaze,ytrain_eye_gaze,sample_weight_value)

    appended_MSE_fusion = np.array(appended_MSE_fusion)
    appended_MSE_pointing = np.array(appended_MSE_pointing)
    appended_MSE_Gaze = np.array(appended_MSE_Gaze)
    
    appended_RMDE_fusion = np.array(appended_RMDE_fusion)
    appended_RMDE_pointing = np.array(appended_RMDE_pointing)
    appended_RMDE_Gaze = np.array(appended_RMDE_Gaze)

    for column in range(len(appended_MSE_fusion)):
        average_fusion_MSE_others = 0
        average_fusion_RMDE_others = 0
        for row in range(len(appended_MSE_fusion)):
            if row == column:
                average_fusion_MSE_per_weight.append(appended_MSE_fusion[row,column])
                average_fusion_RMDE_per_weight.append(appended_RMDE_fusion[row,column])
            else:
                average_fusion_MSE_others += appended_MSE_fusion[row,column]
                average_fusion_RMDE_others += appended_RMDE_fusion[row,column]
        average_fusion_MSE_others = average_fusion_MSE_others / (len(appended_MSE_fusion)-1)
        average_fusion_RMDE_others = average_fusion_RMDE_others / (len(appended_MSE_fusion)-1)    

        average_fusion_MSE_others_total.append(average_fusion_MSE_others)
        average_fusion_RMDE_others_total.append(average_fusion_RMDE_others)
    
    
#     print(average_fusion_MSE_per_weight)
#     print(average_fusion_RMDE_per_weight)

#     print(average_fusion_MSE_others_total)
#     print(average_fusion_RMDE_others_total)
    
    average_fusion_MSE_weights.append(np.mean(average_fusion_MSE_per_weight))
    average_fusion_RMDE_weights.append(np.mean(average_fusion_RMDE_per_weight))
    
    average_fusion_MSE_non_weights_total.append(np.mean(average_fusion_MSE_others_total))
    average_fusion_RMDE_non_weights_total.append(np.mean(average_fusion_RMDE_others_total))
    
    
    np.savetxt('ML-PersRef_ML_Final_dataset_results/appended_MSE_fusion_first_middle_point_another_sample_sample_weight_'+str(sample_weight_value)+'_support_vectors_only.csv', appended_MSE_fusion, delimiter=',')
    np.savetxt('ML-PersRef_ML_Final_dataset_results/appended_MSE_pointing_first_middle_point_another_sample_sample_weight_'+str(sample_weight_value)+'_support_vectors_only.csv', appended_MSE_pointing, delimiter=',')
    np.savetxt('ML-PersRef_ML_Final_dataset_results/appended_MSE_Gaze_first_middle_point_another_sample_sample_weight_'+str(sample_weight_value)+'_support_vectors_only.csv', appended_MSE_Gaze, delimiter=',')
    
    np.savetxt('ML-PersRef_ML_Final_dataset_results/appended_RMDE_fusion_first_middle_point_another_sample_sample_weight_'+str(sample_weight_value)+'_support_vectors_only.csv', appended_RMDE_fusion, delimiter=',')
    np.savetxt('ML-PersRef_ML_Final_dataset_results/appended_RMDE_pointing_first_middle_point_another_sample_sample_weight_'+str(sample_weight_value)+'_support_vectors_only.csv', appended_RMDE_pointing, delimiter=',')
    np.savetxt('ML-PersRef_ML_Final_dataset_results/appended_RMDE_Gaze_first_middle_point_another_sample_sample_weight_'+str(sample_weight_value)+'_support_vectors_only.csv', appended_RMDE_Gaze, delimiter=',')
    